In [4]:
print("ok")

ok


In [5]:
%pwd

'C:\\Users\\Asus\\Downloads\\7f10d54c-18f4-4bf4-b3ca-30be4bd004eb\\End_to_End_Medical_Chatobt\\End_to_End_Medical_Chatobt\\research'

In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'C:\\Users\\Asus\\Downloads\\7f10d54c-18f4-4bf4-b3ca-30be4bd004eb\\End_to_End_Medical_Chatobt\\End_to_End_Medical_Chatobt'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [10]:
extracted_data=load_pdf_file(data='Data')

In [11]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2020-09-21T09:23:51-04:00', 'author': 'Alysa', 'moddate': '2020-09-21T09:23:51-04:00', 'source': 'Data\\Diabetes.pdf', 'total_pages': 84, 'page': 0, 'page_label': '1'}, page_content="Diabetes Mellitus:  \nPathophysiology and Clinical Guidelines \n \nThe Academy of Dental Learning and OSHA Training, LLC, designates this \nactivity for 7 continuing education credits (7 CEs). \n \nBeverly Thomassian, RN, BC-ADM, MPH, CDE \nRevised and Updated by Health Science Editor: Megan Wright, RDH, MS \n \nPublication Date: October 2012 \nUpdated Date: July 2020 \nExpiration Date: August 2023 \n \n \n \nThe Academy of Dental Learning and OSHA Training, LLC is an ADA CERP Recognized \nProvider. ADA CERP is a service of the American Dental Association to assist dental \nprofessionals in identifying quality providers of continuing dental education. ADA CERP does not \napprove or endo

In [12]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 352


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\Asus\AppData\Local\Temp\ipykernel_22120\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\Asus\anaconda3\envs\medibot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\Asus\anaconda3\envs\medibot\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecate

In [ ]:
# pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [21]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,  # your document chunks
    index_name=index_name,
    embedding=embeddings
)

In [22]:
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch

In [29]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [30]:
retrieved_docs = retriever.invoke("What is Diabetes Mellitus?")

In [31]:
retrieved_docs

[Document(metadata={'author': 'Alysa', 'creationdate': '2020-09-21T09:23:51-04:00', 'creator': 'Microsoft® Office Word 2007', 'moddate': '2020-09-21T09:23:51-04:00', 'page': 79.0, 'page_label': '80', 'producer': 'Microsoft® Office Word 2007', 'source': 'Data\\Diabetes.pdf', 'total_pages': 84.0}, page_content='79 \n \nCourse Test: Diabetes Mellitus: Pathophysiology and Clinical \nGuidelines \n1.  Diabetes Mellitus can best be described as: \na.  an endocrine disorder resulting from an inadequate production or impaired  \n     use of insulin. \nb.  a metabolic disorder primarily characterized by loss of protein in the urine. \nc.  an  endocrine  disorder  always  resulting  in  blindness,  kidney  failure,  and  \n     cardiovascular disease. \nd.  an infectious disease process for which there is no cure.'),
 Document(metadata={'author': 'Alysa', 'creationdate': '2020-09-21T09:23:51-04:00', 'creator': 'Microsoft® Office Word 2007', 'moddate': '2020-09-21T09:23:51-04:00', 'page': 6.0, 'pa

In [32]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])


Acromegaly and gigantism are both disorders caused by excessive production of growth hormone. Acromegaly occurs in adults and leads to enlargement of bones and tissues, while gigantism occurs in children and causes excessive growth and height. Both conditions can be caused by tumors in the pituitary gland.


In [36]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])



Stats is a shortened term for "statistics," which refers to the collection, analysis, interpretation, and presentation of numerical data. It is commonly used in the field of healthcare to track and understand trends and patterns in diseases and health outcomes. 
